In [1]:
import sys
sys.path.append('.conda/envs/shear/lib/python3.9/site-packages/')

import numpy as np
import astropy.io.fits as pf
import pylab as mplot
import yaml
import h5py
import healpy as hp

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [7]:
tag = '0613'
project_dir = '/project/chihway/data/decade/'

In [8]:
with h5py.File(project_dir+'metacal_gold_combined_2023'+tag+'.hdf', 'r') as h5r:

    print(h5r.keys())

<KeysViewHDF5 ['BDF_FLUX_ERR_G', 'BDF_FLUX_ERR_I', 'BDF_FLUX_ERR_R', 'BDF_FLUX_ERR_Z', 'BDF_FLUX_G', 'BDF_FLUX_I', 'BDF_FLUX_R', 'BDF_FLUX_Z', 'BDF_S2N', 'BDF_T', 'COADD_OBJECT_ID', 'DEC', 'FLAGS_FOREGROUND', 'FLUXERR_AUTO_G', 'FLUXERR_AUTO_I', 'FLUXERR_AUTO_R', 'FLUXERR_AUTO_Z', 'FLUX_AUTO_G', 'FLUX_AUTO_I', 'FLUX_AUTO_R', 'FLUX_AUTO_Z', 'FLUX_RADIUS_G', 'FLUX_RADIUS_I', 'FLUX_RADIUS_R', 'FLUX_RADIUS_Z', 'Ncutouts_raw', 'RA', 'badfrac', 'id', 'mcal_T_1m', 'mcal_T_1p', 'mcal_T_2m', 'mcal_T_2p', 'mcal_T_noshear', 'mcal_T_ratio_1m', 'mcal_T_ratio_1p', 'mcal_T_ratio_2m', 'mcal_T_ratio_2p', 'mcal_T_ratio_noshear', 'mcal_flags', 'mcal_flux_1m', 'mcal_flux_1p', 'mcal_flux_2m', 'mcal_flux_2p', 'mcal_flux_err_1m', 'mcal_flux_err_1p', 'mcal_flux_err_2m', 'mcal_flux_err_2p', 'mcal_flux_err_noshear', 'mcal_flux_noshear', 'mcal_g_1m', 'mcal_g_1p', 'mcal_g_2m', 'mcal_g_2p', 'mcal_g_cov_1m', 'mcal_g_cov_1p', 'mcal_g_cov_2m', 'mcal_g_cov_2p', 'mcal_g_cov_noshear', 'mcal_g_noshear', 'mcal_g_w', 'mcal_

In [36]:
# first get mask total

with h5py.File(project_dir+'metacal_gold_combined_2023'+tag+'.hdf', 'r') as h5r:
    size_ratio = h5r['mcal_T_ratio_noshear'][:]
    s2n = h5r['mcal_s2n_noshear'][:]
    sg = h5r['sg_bdf'][:] 
    fg = h5r['FLAGS_FOREGROUND'][:] 
    T = h5r['mcal_T_noshear'][:]
    mcal_flags = h5r['mcal_flags'][:]
    g1, g2  = h5r['mcal_g_noshear'][:].T
    flux_r, flux_i, flux_z = h5r['mcal_flux_noshear'][:].T

mag_r = -2.5*np.log10(flux_r)+30
mag_i = -2.5*np.log10(flux_i)+30
mag_z = -2.5*np.log10(flux_z)+30

# PZ mask
mcal_pz_mask = ((mag_i < 23.5) & (mag_i > 18) & 
                    (mag_r < 26)   & (mag_r > 15) & 
                    (mag_z < 26)   & (mag_z > 15) & 
                    (mag_r - mag_i < 4)   & (mag_r - mag_i > -1.5) & 
                    (mag_i - mag_z < 4)   & (mag_i - mag_z > -1.5))

# Metacal cuts based on DES Y3 ones (from here: https://des.ncsa.illinois.edu/releases/y3a2/Y3key-catalogs)
SNR_Mask   = (s2n > 10) & (s2n < 1000)
Tratio_Mask= size_ratio > 0.5
T_Mask = T < 10
Flag_Mask = (mcal_flags == 0)
Other_Mask = np.invert((T > 2) & (s2n < 30)) & np.invert((np.log10(T) < (22.25 - mag_r)/3.5) & (g1**2 + g2**2 > 0.8**2))
SG_Mask = (sg>=4)
FG_Mask = (fg==0)

mask_total = mcal_pz_mask & SNR_Mask & Tratio_Mask & T_Mask & Flag_Mask & Other_Mask & SG_Mask & FG_Mask


/project/chihway/yomori/repo/anaconda3/envs/cosmosis/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in log10
  del sys.path[0]
/project/chihway/yomori/repo/anaconda3/envs/cosmosis/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in log10
  
/project/chihway/yomori/repo/anaconda3/envs/cosmosis/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in log10
  from ipykernel import kernelapp as app
/project/chihway/yomori/repo/anaconda3/envs/cosmosis/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in log10


### get calibration

In [26]:
Mask = {}
for shear_type in ['noshear', '1p', '1m', '2p', '2m']: 
    with h5py.File(project_dir+'metacal_gold_combined_2023'+tag+'.hdf', 'r') as h5r:
        sg = h5r['sg_bdf'][:] 
        fg = h5r['FLAGS_FOREGROUND'][:] 
        T = h5r['mcal_T_'+shear_type][:]
        s2n = h5r['mcal_s2n_'+shear_type][:]
        size_ratio = h5r['mcal_T_ratio_'+shear_type][:]
        mcal_flags = h5r['mcal_flags'][:]
        flux_r, flux_i, flux_z = h5r['mcal_flux_'+shear_type][:].T
        g1, g2  = h5r['mcal_g_'+shear_type][:].T
        
    mag_r = -2.5*np.log10(flux_r)+30
    mag_i = -2.5*np.log10(flux_i)+30
    mag_z = -2.5*np.log10(flux_z)+30

    # PZ mask
    mcal_pz_mask = ((mag_i < 23.5) & (mag_i > 18) & 
                        (mag_r < 26)   & (mag_r > 15) & 
                        (mag_z < 26)   & (mag_z > 15) & 
                        (mag_r - mag_i < 4)   & (mag_r - mag_i > -1.5) & 
                        (mag_i - mag_z < 4)   & (mag_i - mag_z > -1.5))

    # Metacal cuts based on DES Y3 ones (from here: https://des.ncsa.illinois.edu/releases/y3a2/Y3key-catalogs)
    SNR_Mask   = (s2n > 10) & (s2n < 1000)
    Tratio_Mask= size_ratio > 0.5
    T_Mask = T < 10
    Flag_Mask = (mcal_flags == 0)
    Other_Mask = np.invert((T > 2) & (s2n < 30)) & np.invert((np.log10(T) < (22.25 - mag_r)/3.5) & (g1**2 + g2**2 > 0.8**2))
    SG_Mask = (sg>=4)
    FG_Mask = (fg==0)

    mask_total_X = mcal_pz_mask & SNR_Mask & Tratio_Mask & T_Mask & Flag_Mask & Other_Mask & SG_Mask & FG_Mask
    
    Mask[shear_type] = mask_total_X
    del T
    del s2n
    del size_ratio
    del sg
    del fg
    del flux_r, flux_i, flux_z
    del mag_r, mag_i, mag_z

/project/chihway/yomori/repo/anaconda3/envs/cosmosis/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in log10
  del sys.path[0]
/project/chihway/yomori/repo/anaconda3/envs/cosmosis/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in log10
  
/project/chihway/yomori/repo/anaconda3/envs/cosmosis/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in log10
  from ipykernel import kernelapp as app
/project/chihway/yomori/repo/anaconda3/envs/cosmosis/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in log10


In [27]:
def weight_average(values, weights):
    return np.sum(values*weights)/np.sum(weights)

In [28]:
dgamma = 2*0.01
with h5py.File(project_dir+'metacal_gold_combined_2023'+tag+'.hdf', 'r') as h5r:

    R11_w1 =  (weight_average(h5r['mcal_g_1p'][:,0][Mask['noshear']],h5r['mcal_g_w'][Mask['noshear']]) - weight_average(h5r['mcal_g_1m'][:,0][Mask['noshear']], h5r['mcal_g_w'][Mask['noshear']]))/dgamma
    R11s_w1 = (weight_average(h5r['mcal_g_noshear'][:,0][Mask['1p']], h5r['mcal_g_w'][Mask['1p']]) - weight_average(h5r['mcal_g_noshear'][:,0][Mask['1m']], h5r['mcal_g_w'][Mask['1m']]))/dgamma
    R22_w1 =  (weight_average(h5r['mcal_g_2p'][:,1][Mask['noshear']], h5r['mcal_g_w'][Mask['noshear']]) - weight_average(h5r['mcal_g_2m'][:,1][Mask['noshear']], h5r['mcal_g_w'][Mask['noshear']]))/dgamma
    R22s_w1 = (weight_average(h5r['mcal_g_noshear'][:,1][Mask['2p']], h5r['mcal_g_w'][Mask['2p']]) - weight_average(h5r['mcal_g_noshear'][:,1][Mask['2m']], h5r['mcal_g_w'][Mask['2m']]))/dgamma
    R11tot_w1 = R11_w1+R11s_w1
    R22tot_w1 = R22_w1+R22s_w1
    
    print('R11', R11_w1, 'R11s', R11s_w1)
    print('R22', R22_w1, 'R22s', R22s_w1)
    print('R11tot', R11tot_w1, 'R22tot', R22tot_w1)

R11 0.6944438564705219 R11s 0.034199649879673374
R22 0.695467425003138 R22s 0.034768233522071375
R11tot 0.7286435063501953 R22tot 0.7302356585252093


In [32]:
R_w1 = 0.5*(R11tot_w1+R22tot_w1)
print(R_w1)

0.7294395824377022


In [29]:
with h5py.File(project_dir+'metacal_gold_combined_2023'+tag+'.hdf', 'r') as h5r:
    R11_w2 =  (weight_average(h5r['mcal_g_1p'][:,0][Mask['noshear']], h5r['mcal_g_w_v2'][Mask['noshear']]) - weight_average(h5r['mcal_g_1m'][:,0][Mask['noshear']], h5r['mcal_g_w_v2'][Mask['noshear']]))/dgamma
    R11s_w2 = (weight_average(h5r['mcal_g_noshear'][:,0][Mask['1p']], h5r['mcal_g_w_v2'][Mask['1p']]) - weight_average(h5r['mcal_g_noshear'][:,0][Mask['1m']], h5r['mcal_g_w_v2'][Mask['1m']]))/dgamma
    R22_w2 =  (weight_average(h5r['mcal_g_2p'][:,1][Mask['noshear']], h5r['mcal_g_w_v2'][Mask['noshear']]) - weight_average(h5r['mcal_g_2m'][:,1][Mask['noshear']], h5r['mcal_g_w_v2'][Mask['noshear']]))/dgamma
    R22s_w2 = (weight_average(h5r['mcal_g_noshear'][:,1][Mask['2p']], h5r['mcal_g_w_v2'][Mask['2p']]) - weight_average(h5r['mcal_g_noshear'][:,1][Mask['2m']], h5r['mcal_g_w_v2'][Mask['2m']]))/dgamma
    R11tot_w2 = R11_w2+R11s_w2
    R22tot_w2 = R22_w2+R22s_w2
    
    print('R11', R11_w2, 'R11s', R11s_w2)
    print('R22', R22_w2, 'R22s', R22s_w2)
    print('R11tot', R11tot_w2, 'R22tot', R22tot_w2)


R11 0.7443824304923087 R11s 0.047189013967600654
R22 0.7455737932558067 R22s 0.04767254335307269
R11tot 0.7915714444599093 R22tot 0.7932463366088794


In [33]:
R_w2 = 0.5*(R11tot_w2+R22tot_w2)
print(R_w2)

0.7924088905343945


### get area

In [38]:
with h5py.File(project_dir+'metacal_gold_combined_2023'+tag+'.hdf', 'r') as h5r:
    ra = h5r['RA'][mask_total]
    dec = h5r['DEC'][mask_total]
    g1, g2  = h5r['mcal_g_noshear'][:][mask_total].T
    w1 = h5r['mcal_g_w'][mask_total]
    w2= h5r['mcal_g_w_v2'][mask_total]

In [39]:
nside = 4096
map_counts = np.zeros(hp.nside2npix(nside))

phi = ra/180*np.pi
theta = (90.-dec)/180*np.pi

pix = hp.ang2pix(nside, theta, phi)

In [40]:
for i in range(len(pix)):
    map_counts[pix[i]] += 1

In [41]:
area = len(map_counts[map_counts>0])/len(map_counts)*4*np.pi*(180./np.pi)**2*60*60
print('area', area/60/60, 'deg^2')

area 2376.3284069720044 deg^2


In [45]:
# raw number
print(len(ra)/area)

5.534827367047062


### H12 neff and sigmae

In [42]:
neff_H12_w1 = 1./area * (np.sum(w1)**2) / (np.sum(w1**2))
print('neff_H12_w1', neff_H12_w1)

neff_H12_w2 = 1./area * (np.sum(w2)**2) / (np.sum(w2**2))
print('neff_H12_w2', neff_H12_w2)

neff_H12_w1 5.522793378869633
neff_H12_w2 4.5813679430333405


In [43]:
sigmae2_H12_w1 = 0.5*((np.sum(w1**2*(g1/R_w1)**2)/(np.sum(w1))**2)+(np.sum(w1**2*(g2/R_w1)**2)/(np.sum(w1))**2))*(area*neff_H12_w1)
print('sigmae_H12_w1', sigmae2_H12_w1**0.5)

sigmae2_H12_w2 = 0.5*((np.sum(w2**2*(g1/R_w2)**2)/(np.sum(w2))**2)+(np.sum(w2**2*(g2/R_w2)**2)/(np.sum(w2))**2))*(area*neff_H12_w2)
print('sigmae_H12_w2', sigmae2_H12_w2**0.5)

sigmae_H12_w1 0.27015235124431114
sigmae_H12_w2 0.2376053038749086


### C13 neff and sigmae

In [46]:
with h5py.File(project_dir+'metacal_gold_combined_2023'+tag+'.hdf', 'r') as h5r:
    mcal_g_cov = h5r['mcal_g_cov_noshear'][:][mask_total]

In [47]:
mcal_g_cov.shape

(47349243, 2, 2)

In [48]:
sigma2_e_m = (mcal_g_cov[:,0,0] + mcal_g_cov[:,1,1])
print(sigma2_e_m)

[0.00025132 0.00111886 0.00081327 ... 0.00076986 0.00096072 0.00839997]


In [49]:
sigmae2_C13_w1 = 0.5* np.sum(w1**2*((g1/R_w1)**2+(g2/R_w1)**2 - sigma2_e_m/R_w1**2)) / np.sum(w1**2)
print('sigmae_C13_w1', sigmae2_C13_w1**0.5)

sigmae2_C13_w2 = 0.5* np.sum(w2**2*((g1/R_w2)**2+(g2/R_w2)**2 - sigma2_e_m/R_w2**2)) / np.sum(w2**2)
print('sigmae_C13_w2', sigmae2_C13_w2**0.5)

sigmae_C13_w1 0.26448271637393356
sigmae_C13_w2 0.23491319200660665


In [51]:
neff_C13_w1 = 1./area * (np.sum(w1)**2) / (np.sum(w1**2))*(np.sum(w1**2*((g1/R_w1)**2+(g2/R_w1)**2 - sigma2_e_m/R_w1**2)))/ np.sum(w1**2*((g1/R_w1)**2+(g2/R_w1)**2))
print('neff_C13_w1', neff_C13_w1)

neff_C13_w2 = 1./area * (np.sum(w2)**2) / (np.sum(w2**2))*(np.sum(w2**2*((g1/R_w2)**2+(g2/R_w2)**2 - sigma2_e_m/R_w2**2)))/ np.sum(w2**2*((g1/R_w2)**2+(g2/R_w2)**2))
print('neff_C13_w2', neff_C13_w2)

neff_C13_w1 5.293414289453894
neff_C13_w2 4.478140581630089
